In [1]:
#########################################
# 17th of January 2017
# Alexandre Dos Santos and Amélie Simon
# To clean Tel Jeune SMS -> Return one file with all messages of a user appended and the subject conversation code
# same codes used for emails
#########################################

import csv
import pickle
import html
from time import time

t0 = time()

encoding = 'utf-8'
path_to_data = './data'
#Ouverture du fichier de sms
path = path_to_data+"movies-metadata.csv"
sms = []

with open(path, newline='',encoding = encoding ) as csvfile:
        sms_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in sms_reader:
            sms.append(row)

## Build conversations

In [2]:
from tqdm import tqdm

#Ouverture du fichier de conversations et contruction du dataset
path = path_to_data+"app_conversations.csv"
conversations = []
subjects = []
broke_conv_ids = []
conv_ids = set() #liste des id de conv. déjà ouvertes
length = 0

with open(path, newline='' ,encoding = encoding ) as csvfile:
        conv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in conv_reader:
            length+=1

            
to_standard_subject = dict()
to_standard_subject[''] = []
with open("./data/subject_sms.csv", newline='',encoding = encoding ) as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in csv_reader:
            to_standard_subject[row[0]] = row[1:]

with open(path, newline='' ,encoding = encoding ) as csvfile:
        conv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in conv_reader:
            break
        for row in tqdm(conv_reader):            
            conv_id = int(row[0]) - 1    #Id de conversation
            subject = row[3]         #Sujet de la conversation
            subject_id = row[4]      #Id du sujet de la conversation
            sms_index = int(row[5])  #Index du sms concerné
            sender = sms[sms_index][1]
            sender_id = sms[sms_index][2]
            
            # Si la conversation existe déjà
            if conv_id in conv_ids:
                conversation = conversations[conv_id]
                if sender == 'mo': #Si l'envoyeur du message est le jeune
                    if len(conversation[-1][1]) == 0: # Si le message est une continuation du message précédent
                        conversation[-1][0].append(sms_index)
                    else:
                        conversation.append([[sms_index], []])
                else: #Si l'envoyeur du message est l'intervenant 
                    if len(conversation[-1][1]) == 0: # Si le message est une réponse directe
                        conversation[-1][1] = [sms_index]
                    else:
                        conversation[-1][1].append(sms_index)
            else:
                if sender == 'mo':
                    conv_ids.add(conv_id) #Rajouter l'id de conversation à la liste des id de conv. déjà ouvertes
                    conversations.append([[[sms_index], []]])
                    subjects.append(to_standard_subject[subject])
                else: #Si l'envoyeur du premier message de la conversation est l'intervenant 
                    broke_conv_ids.append(conv_id)
                    for j in range(len(sms[:sms_index])):
                        if sender_id == sms[sms_index-j][3]:
                            first_message_id = [sms_index-j]
                            conv_ids.add(conv_id) #Rajouter l'id de conversation à la liste des id de conv. déjà ouvertes
                            conversations.append([[first_message_id, [sms_index]]])
                            subjects.append(to_standard_subject[subject])
                            break

825994it [00:21, 38398.10it/s]


## Build Q&A data structure

In [3]:
import re

def replace_ponctuation(raw_text):
    raw_text = raw_text.replace("...", "&&&")
    raw_text = raw_text.replace(".", " . ").replace(",", " , ").replace(";", " ; ").replace(":", " : ").replace("!", " ! ").replace("?", " ? ").replace("&&&", " &&& ").replace("(", " ( ").replace(")", " ) ").replace('"', ' " ')
                        
    return re.sub(' +', " ", raw_text).replace("&&&", "...")

In [4]:
Q_A = []
line_no = 0
for j, conversation in enumerate(tqdm(conversations)):
    context = []
    for line in conversation:
        splited_question = [html.unescape(sms[i][4]) for i in line[0]]
        if len(line[0]) > 1 and sms[line[0][1]][4][0] != ' ':
            joint = ' '
        else:
            joint = ''
        question = replace_ponctuation(joint.join(splited_question))
        answer = replace_ponctuation(' '.join([html.unescape(sms[i][4]) for i in line[1]]))
        if len(answer) > 0:
            Q_A.append([context[:], question, answer, subjects[j]])
            context.append(line_no)
            line_no += 1 

100%|██████████████████████████████████████████████████████████████████████████| 31017/31017 [00:17<00:00, 1776.56it/s]


## Tokenize

In [5]:
from multiprocessing import Pool
from utils import custom_tokenizer

tokenize = custom_tokenizer.init(use_nltk_tok=True, use_stemmer=False)

questions = [line[1] for line in Q_A]
answers = [line[2] for line in Q_A]
tokenized_questions = []
tokenized_answers = []

p = Pool()
tokenized_questions = p.map(tokenize, questions)
tokenized_answers = p.map(tokenize, answers)
p.close()

for i, line in enumerate(Q_A):
    line[1] = tokenized_questions[i]
    line[2] = tokenized_answers[i]

In [6]:
import spacy
tokenize = spacy.load('fr')

In [7]:
for question in questions[:100]:
    doc = tokenize(question)
#     print([(token.text, token.lemma_, token.pos_, token.tag_) for token in doc])
    print(doc)
    print([(ent.text, ent.label_) for ent in doc.ents])
    

Icône
[('Icône', 'LOC')]
Comment vous savez mon nom ? ? ? 
[]
Ah bon , on a plus de secret maintenant
[]
Allo Ok merci d'être là : ) 
[]
Je me sens toute seule
[]
Varennes , Montréal , Canada
[('Varennes', 'LOC'), ('Montréal', 'LOC'), ('Canada', 'LOC')]
Car j'avais une amie qui se nomme Marie-Maude puis nous ne sommes plus amies car elle se fâchait régulièrement envers moi . Elle dit qu'elle a une tête de cochon . Puis quand je lui met en pleine face que c'est ça la réalité elle ne comprend pas . Puis elle fait des crises de colère . Mais moi je dis seulement que je suis gentille avec elle . 
[('Marie', 'PER'), ('Maude', 'PER')]
J'aimerais que vous m'aidiez en me disant qu'elle serait la bonne chose à faire ? 
[]
Passer à autre chose car ce n'est pas la première fois que elle me fait le coup . 
[]
Oui mais on habite l'une près de l'autre
[]
Oui mais une veut m'arracher la tête Par ce qu'elle est en furie je l'as connais depuis qu'elle a 10 ans aussi ses parents sont séparés . elle est 

## Build vocabulary

In [8]:
import itertools
from collections import Counter

words_questions = list(itertools.chain.from_iterable(tokenized_questions))
words_answers = list(itertools.chain.from_iterable(tokenized_answers))

vocab = [('<RAR>',999999), ('<SOS>',999999), ('<EOS>',999999), ('<PAD>',999999), ('<NAME>',999999)] + Counter(words_questions+words_answers).most_common()
del words_questions, words_answers

## Anonymization and vocab trimming

In [9]:
possible_Names = []

for question in tokenized_questions:
    if len(question) > 1:
        for word in question[1:]:
            if word[0].isupper():
                possible_Names.append(word)

In [10]:
import csv

min_count = 5
counts = [line[1] for line in vocab]

possible_Names_sorted = [word for word, count in Counter(possible_Names).most_common()]
possible_names_counts = sorted(Counter(possible_Names).values())[::-1]

possible_Names_ = [possible_Names_sorted[i] for i, count in enumerate(possible_names_counts) if count >= min_count]

# UNCOMMENT FOR SAVING

# with open("possible_names.csv", "w", encoding='utf-8', newline='') as f:
#     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
#     for name in possible_Names_:
#         csv_writer.writerow([name])

In [11]:
names = []

with open("./data/possible_names.csv", "r", encoding='utf-8', newline='') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for line in csv_reader:
        if len(line) > 1:
            if line[1] == '1':
                names.append(line[0].lower())     
print(len(names))

398


### Update vocab with lowercases 

In [12]:
tmp = vocab[:5]
for word, count in vocab[5:]:
    tmp.append((word.lower(),count))
print(len(tmp))
    
vocab_lower_doublon = [word for word,count in Counter([word for word, count in tmp]).most_common() if count>1]

vocab = []
for j, (word, count) in enumerate(tqdm(tmp)):
    if word in vocab_lower_doublon:
        try:
            vocab_lower_words = [word for word, count in vocab]
            i = vocab_lower_words.index(word)
            vocab[i] = (word,count+vocab[i][1])
        except:
            vocab.append((word, count))
    else:
        vocab.append((word, count))

114792


100%|█████████████████████████████████████████████████████████████████████████| 114792/114792 [02:52<00:00, 666.50it/s]


### Save Vocab

In [13]:
with open('./data/vocab.csv', 'w', newline='', encoding=encoding) as csvfile:
    vocab_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for (word, count) in vocab:
        vocab_writer.writerow([word,count])

## Index Q_A with vocab indexer

In [14]:
vocab_indexer = dict()
for i, (word, count) in enumerate(vocab):
    if count >= min_count:
        if not word in names:
            vocab_indexer[word] = i
        else:
            vocab_indexer[word] = 4
    else:
        vocab_indexer[word] = 0

In [15]:
Q_A_indexed = []

for i, (context, question, answer, subject) in enumerate(Q_A):
    question_indexed = [vocab_indexer[word.lower()] for word in question]
    answer_indexed = [vocab_indexer[word.lower()] for word in answer]
    standard_subject = subject[:]
    if len(subject) < 2:
        standard_subject = ['', '']
    Q_A_indexed.append([context, question_indexed, answer_indexed, standard_subject])

In [14]:
for context, question, answer, subject in Q_A_indexed[:10]:
    print('QUESTION:', ' '.join([vocab[index][0] for index in question]))
    print('ANSWER:', ' '.join([vocab[index][0] for index in answer]))
    print('SUBJECT: ', subject)

QUESTION: <RAR>
ANSWER: bonjour comment puis-je t'aider <RAR>
SUBJECT:  ['', '']
QUESTION: comment vous savez mon nom ? ? ?
ANSWER: c'est écrit dans mon écran : )
SUBJECT:  ['', '']
QUESTION: ah bon , on a plus de secret maintenant
ANSWER: ça a l'air que non ! bonne chance
SUBJECT:  ['', '']
QUESTION: allo ok merci d'être là : )
ANSWER: allo bon matin , le texto est maintenant ouvert . nous sommes la jusqu , a 23h si tu veux discuter avec nous : )
SUBJECT:  ['Ça va pas', '']
QUESTION: je me sens toute seule
ANSWER: pour commencer , accepterais tu de me donner ton age et ta ville ? c pour nos stats , mais ca m'aide tjrs a me situer dans la conversation : )
SUBJECT:  ['Ça va pas', '']
QUESTION: <NAME> , <NAME> , <NAME>
ANSWER: merci : ) pourrais tu m'en dire un peu plus sur ta situation ? pourquoi te sens-tu seule ?
SUBJECT:  ['Ça va pas', '']
QUESTION: car j'avais une amie qui se nomme <RAR> puis nous ne sommes plus amies car elle se fâchait régulièrement envers moi . elle dit qu'elle a

# Save pickle

In [16]:
path = './data/Q_A.npy'
with open(path, 'wb') as file:
    pickle.dump(Q_A_indexed, file)

### loading snippet

## Create Conversation csv file

In [18]:
import csv
import pickle

vocab_indexer = dict()
vocab = []
counts = []

with open('./data/vocab.csv', 'r', newline='', encoding='utf-8') as csvfile:
    vocab_reader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    i = 0
    for word, count in vocab_reader:
        vocab_indexer[word] = i
        vocab.append(word)
        counts.append(count)
        i += 1

with open('./data/Q_A.npy', 'rb') as file:
    Q_A = pickle.load(file)

questions = []
answers = []
contextes = []
subjects = []

for i, line in enumerate(Q_A):
    if len(line[2]) > 0:
        questions.append([vocab[index] for index in line[1][:]])
        answers.append([vocab[index] for index in line[2][:]])
        contextes.append(line[0][:])
        subjects.append(line[3][:])
    else:
        print('ERROR')
        raise

In [18]:
import json
import csv
import pickle

vocab_indexer = dict()
vocab = []
counts = []

with open('./data/vocab.csv', 'r', newline='', encoding='utf-8') as csvfile:
    vocab_reader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    i = 0
    for word, count in vocab_reader:
        vocab_indexer[word] = i
        vocab.append(word)
        counts.append(count)
        i += 1

with open('./data/Q_A.npy', 'rb') as file:
    Q_A = pickle.load(file)

convos = []
conv_no = 0
convo = []

for i, line in enumerate(Q_A[:100]):
    if len(line[2]) > 0:
        if len(line[0]) == 0:
            conv_no += 1;
            convos.append(convo)
            convo = []
        else:
            question = ' '.join([vocab[index] for index in line[1][:]])
            answer = ' '.join([vocab[index] for index in line[2][:]])
            convo.append({"question":question, "answer":answer})
    else:
        print('ERROR')
        raise
convos.append(convo)
print(conv_no)
with open("./convos.json", 'w', encoding='utf-8') as f: 
    json.dump(convos, f, sort_keys=True, indent=4)

12


In [26]:
conv_no = -1

with open('./data/conversations.csv', 'w', newline='', encoding='utf-8') as csvfile:
    conv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for i, line in enumerate(questions):
        if len(contextes[i]) == 0:
            conv_no+=1
        question = ' '.join([word for word in line])
        answer = ' '.join([word for word in answers[i]])
        conv_writer.writerow([conv_no, question, answer, subjects[i]])

## Create file for classification

In [1]:
import csv
import pickle

vocab_indexer = dict()
vocab = []
counts = []

with open('./data/vocab.csv', 'r', newline='', encoding='utf-8') as csvfile:
    vocab_reader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    i = 0
    for word, count in vocab_reader:
        vocab_indexer[word] = i
        vocab.append(word)
        counts.append(count)
        i += 1

with open('./data/Q_A.npy', 'rb') as file:
    Q_A = pickle.load(file)

questions = []
contextes = []
subjects = []

for i, line in enumerate(Q_A):
    if len(line[2]) > 0:
        contextes.append(line[0][:])
        questions.append([vocab[index] for index in line[1][:]])
        subjects.append(line[3][:])
    else:
        print('ERROR')
        raise

In [2]:
from collections import Counter

subject_to_int = list(Counter([line[0] for line in subjects]))
print(subject_to_int)

['', 'Ça va pas', 'Amour', 'Drogues et alcool', 'Sexualité', 'Amis et famille', 'Technos', 'Academic life', 'Bien-être', 'Social health', ' ']


In [29]:
conv_no = -1
whole_question = ''
subject = ['', '']

with open('./data/data_fr.csv', 'w', newline='', encoding='utf-8') as csvfile:
    conv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for i, line in enumerate(questions):
        question = ' '.join([word for word in line])
        if len(contextes[i]) == 0:
            conv_no+=1
            if whole_question == '':
                whole_question = question
            data = [whole_question]
            data.extend(subject)
            conv_writer.writerow(data)
            whole_question = question
            subject = subjects[i]
        elif len(contextes[i]) < 5:
            whole_question += (' '+question)
        else:
            conv_writer.writerow([question, '', ''])

In [7]:
from nltk.stem.snowball import FrenchStemmer
from tqdm import tqdm

stemmer = FrenchStemmer(ignore_stopwords=False)

stemmed_questions = []

for question in tqdm(questions):
    words = [stemmer.stem(token) for token in question]
    stemmed_questions.append(words)

100%|████████████████████████████████████████████████████████████████████████| 265917/265917 [01:46<00:00, 2488.72it/s]


In [11]:
conv_no = -1
whole_question = ''
subject = ['', '']

with open('./data/stemmed_data_fr.csv', 'w', newline='', encoding='utf-8') as csvfile:
    conv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for i, line in enumerate(stemmed_questions):
        question = ' '.join([word for word in line])
        if len(contextes[i]) == 0:
            conv_no+=1
            if whole_question == '':
                whole_question = question
            data = [whole_question]
            data.extend(subject)
            conv_writer.writerow(data)
            whole_question = question
            subject = subjects[i]
        elif len(contextes[i]) < 5:
            whole_question += (' '+question)
        else:
            conv_writer.writerow([question, '', ''])
            
import itertools
from collections import Counter

words_questions = list(itertools.chain.from_iterable(stemmed_questions))

vocab = [('<RAR>',999999), ('<SOS>',999999), ('<EOS>',999999), ('<PAD>',999999), ('<NAME>',999999)] + Counter(words_questions).most_common()
del words_questions

with open('./data/stemmed_vocab.csv', 'w', newline='', encoding='utf-8') as csvfile:
    vocab_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for (word, count) in vocab:
        vocab_writer.writerow([word,count])

# Save CSV

In [ ]:
path = './data/Q_A.csv'
bar = progressbar.ProgressBar(max_value=len(Q_A))

with open(path, 'w', newline='', encoding=encoding) as csvfile:
    conv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for i, line in enumerate(Q_A):
        if i % int(len(Q_A)/100) == 0:
            bar.update(i)
        conv_writer.writerow(line)

In [1]:
subjects = []
import csv

with open("./data/subject_sms.txt", encoding='ANSI') as f:
    for line in f:
        elems = line.split("\t")
        subject = [elems[0], elems[1], elems[2][:-1]]
        subjects.append(subject)      
with open('./data/subject_sms.csv', 'w', newline='', encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for line in subjects:
        csv_writer.writerow(line)